# Day 23
## Part 1

In [1]:
from collections import defaultdict, namedtuple
import pyrsistent as pyr
from heapq import heapify, heappop, heappush

Edge = namedtuple('Edge', 'neighbour, length')

def create_graph():
    g = defaultdict(list)
    top_row = [1,2,4,6,8,10,11]
    for i in range(len(top_row) - 1):
        g[str(top_row[i])].append(Edge(str(top_row[i + 1]), top_row[i + 1] - top_row[i]))
        g[str(top_row[i + 1])].append(Edge(str(top_row[i]), top_row[i + 1] - top_row[i]))
    for xs, y in [
        (("2", "4"), "A1"),
        (("4", "6"), "B1"),
        (("6", "8"), "C1"),
        (("8", "10"), "D1")
    ]:
        for x in xs:
            g[x].append(Edge(y, 2))
            g[y].append(Edge(x, 2))
    for c in "ABCD":
        g[f"{c}1"].append(Edge(f"{c}2", 1))
        g[f"{c}2"].append(Edge(f"{c}1", 1))
    return g

def parse_data(s):
    lines = s.strip().splitlines()
    room_locations = {
        "A1": (2, 3),
        "A2": (3, 3),
        "B1": (2, 5),
        "B2": (3, 5),
        "C1": (2, 7),
        "C2": (3, 7),
        "D1": (2, 9),
        "D2": (3, 9)
    }
    burrow = {}
    for room in room_locations:
        row, col = room_locations[room]
        burrow[room] = lines[row][col]
    return burrow

test_data = parse_data('''
#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########
''')

test_data

{'A1': 'B',
 'A2': 'A',
 'B1': 'C',
 'B2': 'D',
 'C1': 'B',
 'C2': 'C',
 'D1': 'D',
 'D2': 'A'}

In [7]:
from functools import total_ordering
from dataclasses import dataclass

@total_ordering
@dataclass
class Burrow:
    energy: int
    positions: dict
        
    def __eq__(self, other):
        return self.energy == other.energy
    
    def __lt__(self, other):
        return self.energy < other.energy 
    
    def __iter__(self):
        return iter((self.energy, self.positions))
    

def possible_path(node, graph, positions):
    # All possible paths, whether or not they are legitimate
    paths = []
    seen = set()
    to_see = [(0, node)]
    amphipod = positions[node]
    energy_per_step = {'A': 1, 'B': 10, 'C': 100, 'D': 1000}[amphipod] 
    while to_see:
        energy_so_far, this_node = heappop(to_see)
        if this_node not in seen:
            seen.add(this_node)
            for edge in graph[this_node]:
                if edge.neighbour not in positions and edge.neighbour not in seen:
                    paths.append((edge.neighbour, energy_so_far + edge.length * energy_per_step))
                    heappush(to_see, (energy_so_far + edge.length * energy_per_step, edge.neighbour))
              
    # Which paths are legitimate?
    for next_node, energy in paths:
        node_is_hallway = node.isdigit()
        next_node_is_hallway = next_node.isdigit()
        amphipods_room = next_node[0] == amphipod
        amphipods_room_clear = all(
            positions.get(f'{amphipod}{c}', amphipod) == amphipod
            for c in '12'
        )
        already_in_final_room = node[0] == amphipod
        legit = False
        if not node_is_hallway and next_node_is_hallway:
            legit = True
        if amphipods_room and amphipods_room_clear:
            legit = True
        if already_in_final_room and amphipods_room_clear and next_node_is_hallway:
            legit = False
        if legit:
            yield next_node, energy
            

def finished(positions):
    return all(
        p[0] == positions[p] for p in positions
    )

def part_1(graph, positions):
    queue = [(0, positions)]
    seen = set()
    while queue:
        energy_so_far, pos_so_far = heappop(queue)
        if pyr.pmap(pos_so_far) not in seen:
            seen.add(pyr.pmap(pos_so_far))
            if finished(pos_so_far):
                return energy_so_far
            for node in pos_so_far:
                amphipod = pos_so_far[node]
                for next_node, next_energy in possible_path(node, graph, pos_so_far):
                    next_pos = pos_so_far.copy()
                    del next_pos[node]
                    next_pos[next_node] = amphipod
                    if pyr.pmap(next_pos) not in seen:
                        heappush(
                            queue,
                            Burrow(energy_so_far + next_energy,
                                   next_pos)
                        )

graph = create_graph()

In [9]:
part_1(graph, test_data)

12521